In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
decoder_layer = model.model.layers[0]
decoder_layer

/project/michelangelo_users/software/rajath.rao/miniconda3/envs/cuda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear(in_features=2048, out_features=512, bias=False)
    (v_proj): Linear(in_features=2048, out_features=512, bias=False)
    (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
    (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)

In [2]:
from pprint import pprint
model.config._attn_implementation = "eager"
pprint(model.config)

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.48.3",
  "use_cache": true,
  "vocab_size": 128256
}



In [3]:
# Prepare inputs
device = 'cpu'
input_ids = torch.randint(0, model.config.vocab_size, (1, 512))
input_embeds = model.model.embed_tokens(input_ids).to(device)
cache_position = torch.arange(0, input_ids.shape[1], dtype=torch.long).to(device)
position_ids = cache_position.unsqueeze(0).to(device)
causal_mask = model.model._update_causal_mask(
    torch.ones_like(input_ids), input_embeds, cache_position, None, False
)
position_embeddings = model.model.rotary_emb(input_embeds, position_ids)

hidden_states = decoder_layer(
    input_embeds,
    attention_mask=causal_mask,
    position_ids=position_ids,
    past_key_value=None,
    output_attentions=False,
    use_cache=False,
    cache_position=cache_position,
    position_embeddings=position_embeddings,
)[0]
print(hidden_states.shape)

torch.Size([1, 512, 2048])
